# **Task 3**

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [75]:
pd.options.mode.chained_assignment = None

## **Data importing and cleaning**

In [4]:
# import all necessary datasets
data_orders_payments = pd.read_csv('data_raw/order_payments.csv')
data_orders = pd.read_csv('data_raw/orders.csv')
data_customers = pd.read_csv('data_raw/customers.csv')
data_sellers = pd.read_csv('data_raw/sellers (1).csv')
data_order_items = pd.read_csv('data_raw/order_items.csv')
data_product = pd.read_csv('data_raw/products.csv')
data_geolocation = pd.read_csv('data_raw/geolocation.csv')

### **Prepare geolocation, merge with customers and orders**

In [5]:
# Take a look at customers
data_customers.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [6]:
# A lot of cities, this could not be a feature in the future
# But we could use this info to compare seller and customer city
data_customers.customer_city.value_counts()

sao paulo              15540
rio de janeiro          6882
belo horizonte          2773
brasilia                2131
curitiba                1521
                       ...  
vitoria das missoes        1
pedra lavrada              1
terenos                    1
santana do capivari        1
barao                      1
Name: customer_city, Length: 4119, dtype: int64

In [7]:
# A lot of states, this could be a feature in the future
# But we could use this info to compare seller and customer state
data_customers.customer_state.value_counts()

SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: customer_state, dtype: int64

In [8]:
data_geolocation.head(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP


In [9]:
data_customers.shape, data_geolocation.shape

((99441, 5), (1000163, 5))

In [10]:
# We have 278 missing values with coordinates in our dataset. 
# It could be useful in feature engineering to calculate the distance between seller and customer
# So we need only raws with lat and lng
data_customers['customer_zip_code_prefix'].isin(data_geolocation['geolocation_zip_code_prefix']).value_counts()

True     99163
False      278
Name: customer_zip_code_prefix, dtype: int64

In [11]:
# As we have mininum 100 examples for each geolocation zip code prefix and out cliet approved the 
# aggregation, we could group values by mean()
data_geolocation[data_geolocation['geolocation_zip_code_prefix'] == 14409]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
334422,14409,-20.509897,-47.397866,franca,SP
334449,14409,-20.497396,-47.399241,franca,SP
334459,14409,-20.510459,-47.399553,franca,SP
334462,14409,-20.480940,-47.394161,franca,SP
334463,14409,-20.515413,-47.398194,franca,SP
...,...,...,...,...,...
336625,14409,-20.483960,-47.403324,franca,SP
336650,14409,-20.483960,-47.403324,franca,SP
336653,14409,-20.487255,-47.405925,franca,SP
336657,14409,-20.510459,-47.399553,franca,SP


In [14]:
# Check the number of the unique zip codes
data_geolocation.geolocation_zip_code_prefix.unique().shape

(19015,)

In [18]:
data_geolocation_agg = data_geolocation.groupby('geolocation_zip_code_prefix').agg({'geolocation_lat':'mean',
                                                                                    'geolocation_lng':'mean'})
data_geolocation_agg

,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,,
1001,-23.550190,-46.634024
1002,-23.548146,-46.634979
1003,-23.548994,-46.635731
1004,-23.549799,-46.634757
1005,-23.549456,-46.636733
...,...,...
99960,-27.953722,-52.025511
99965,-28.183372,-52.039850
99970,-28.343766,-51.874689


In [25]:
# Now its time to merge geolocation with customers
# 

data_merge_0 = pd.merge(data_customers, 
                        data_geolocation_agg, 
                        how='inner', 
                        left_on = 'customer_zip_code_prefix', 
                        right_on = 'geolocation_zip_code_prefix').reset_index()

# rename lat and lng because we will have similar from sellers dataframe
data_merge_0.rename(columns={'geolocation_lat':'customer_lat', 'geolocation_lng': 'customer_lng'}, inplace=True)

data_merge_0.drop(['customer_zip_code_prefix','customer_unique_id','index'], axis=1, inplace=True)

data_merge_0.head(2)

,customer_id,customer_city,customer_state,customer_lat,customer_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP,-20.498489,-47.396929
1,5dca924cc99eea2dc5ba40d11ec5dd0f,franca,SP,-20.498489,-47.396929


In [27]:
# Check the shape of the final dataset
data_merge_0.shape

(99163, 5)

In [38]:
data_orders.order_status.value_counts().sort_values(ascending=False)

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [39]:
# As we could see everything except delivered has no delivered date.
# So, that was not a necessary data for us
data_orders[data_orders.order_status == 'shipped'].head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaN,2018-06-28 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaN,2018-02-07 00:00:00


In [47]:
# Create mask and drop all unnecesary values
mask_not_delivered = data_orders.order_status.unique()[1:]
data_orders.drop(data_orders[data_orders.order_status.isin(mask_not_delivered)].index, axis=0, inplace=True)
data_orders.shape

(96478, 8)

In [51]:
# merge of data with orders and and customers
data_merge_1 = pd.merge(data_orders, data_merge_0, on='customer_id')

data_merge_1.drop(['order_status'], axis=1, inplace=True)

data_merge_1.head(2)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,customer_lat,customer_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,sao paulo,SP,-23.576983,-46.587161
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,barreiras,BA,-12.177924,-44.660711


### **Merge sellers with geolocation, then with order_item and product**

In [55]:
# Check sellers shape
data_sellers.shape

(3095, 4)

In [53]:
# Prepare sellers and merge with geolocation
data_merge_1_5 = pd.merge(data_sellers, data_geolocation_agg, 
                          left_on = 'seller_zip_code_prefix', 
                          right_on='geolocation_zip_code_prefix')

data_merge_1_5.rename(columns={'geolocation_lat':'seller_lat', 'geolocation_lng': 'seller_lng'}, inplace=True)

data_merge_1_5.drop(['seller_zip_code_prefix'], axis=1, inplace=True)

data_merge_1_5

,seller_id,seller_city,seller_state,seller_lat,seller_lng
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,-22.893848,-47.061337
1,e0eabded302882513ced4ea3eb0c7059,campinas,SP,-22.893848,-47.061337


In [57]:
# Merge order items and prepared sellers with geolocation
data_merge_2 = pd.merge(data_order_items, data_merge_1_5, on='seller_id')

data_merge_2.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,seller_lat,seller_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,volta redonda,SP,-22.496953,-44.127492
1,0188777fe321843a18be24a6e9aa1e53,1,436c8d57ff8d4aa254318e9bd9b48c83,48436dade18ac8b2bce089ec2a041202,2017-07-31 14:35:11,55.9,9.94,volta redonda,SP,-22.496953,-44.127492


In [59]:
data_merge_3 = pd.merge(data_merge_2, data_product, on='product_id')
data_merge_3.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,seller_lat,seller_lng,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,volta redonda,SP,-22.496953,-44.127492,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,volta redonda,SP,-22.496953,-44.127492,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0


In [62]:
del data_merge_0, data_merge_1_5, data_merge_2

In [67]:
data_merge_1.head(2)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,customer_lat,customer_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,sao paulo,SP,-23.576983,-46.587161
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,barreiras,BA,-12.177924,-44.660711


In [60]:
data_merge_1.shape 

(96214, 11)

In [68]:
data_merge_3.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,seller_lat,seller_lng,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,volta redonda,SP,-22.496953,-44.127492,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,volta redonda,SP,-22.496953,-44.127492,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0


In [61]:
data_merge_3.shape 

(112397, 19)

In [75]:
data_merge_4 = pd.merge(data_merge_1, data_merge_3, how='inner', on='order_id')
data_merge_4.dropna(inplace=True)
data_merge_4.head(2)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,customer_lat,customer_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,seller_lat,seller_lng,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,sao paulo,SP,-23.576983,-46.587161,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,maua,SP,-23.680729,-46.444238,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,barreiras,BA,-12.177924,-44.660711,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,belo horizonte,SP,-19.807681,-43.980427,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0


In [76]:
data_merge_4.shape

(108106, 29)

## **Preprocessing**

In [4]:
# Count of items in order
# As a result....Probably we have to agg orders
data_merge_4.order_item_id.value_counts()

1     94624
2      9458
3      2207
4       923
5       435
6       247
7        58
8        35
9        28
10       25
11       17
12       13
13        8
14        7
15        5
16        3
17        3
18        3
19        3
20        3
21        1
Name: order_item_id, dtype: int64

In [5]:
# Check the count number of different products on one order, for example for the biggest
data_merge_4[data_merge_4.order_item_id == 21]

,Unnamed: 0,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,customer_lat,customer_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,seller_lat,seller_lng,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
96482,97856,8272b63d03f5f79c56e9e4120aec44ef,fc3d1daec319d62d49bfb5e1f83123e9,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28 00:00:00,sao paulo,SP,-23.680743,-46.784224,21,79ce45dbc2ea29b22b5a261bbb7b7ee7,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,7.8,6.57,sao paulo,SP,-23.488234,-46.469989,beleza_saude,27.0,152.0,2.0,1000.0,25.0,6.0,12.0


In [10]:
data_merge_4[data_merge_4.order_id == '8272b63d03f5f79c56e9e4120aec44ef'].head(2)

,Unnamed: 0,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,customer_lat,customer_lng,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,seller_lat,seller_lng,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
96462,97836,8272b63d03f5f79c56e9e4120aec44ef,fc3d1daec319d62d49bfb5e1f83123e9,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28 00:00:00,sao paulo,SP,-23.680743,-46.784224,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,sao paulo,SP,-23.488234,-46.469989,beleza_saude,45.0,232.0,3.0,800.0,21.0,4.0,15.0
96463,97837,8272b63d03f5f79c56e9e4120aec44ef,fc3d1daec319d62d49bfb5e1f83123e9,2017-07-16 18:19:25,2017-07-17 18:25:23,2017-07-20 15:45:53,2017-07-31 18:03:02,2017-07-28 00:00:00,sao paulo,SP,-23.680743,-46.784224,12,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89,sao paulo,SP,-23.488234,-46.469989,beleza_saude,45.0,232.0,3.0,800.0,21.0,4.0,15.0


In [8]:
data_merge_4[data_merge_4.order_id == '8272b63d03f5f79c56e9e4120aec44ef'].product_id.value_counts()

270516a3f41dc035aa87d220228f844c    10
05b515fdc76e888aada3c6d66c201dff    10
79ce45dbc2ea29b22b5a261bbb7b7ee7     1
Name: product_id, dtype: int64

In [103]:
# save results
data_merge_4.to_csv('data_raw/data_merged.csv')

In [52]:
# import previous results 
data_merge_4  = pd.read_csv('data_raw/data_merged.csv')

In [53]:
# We are going to rearange columns for convinience
data_merge_4.columns.to_list()

['Unnamed: 0',
 'order_id',
 'customer_id',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'customer_city',
 'customer_state',
 'customer_lat',
 'customer_lng',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value',
 'seller_city',
 'seller_state',
 'seller_lat',
 'seller_lng',
 'product_category_name',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [87]:
data_merge = data_merge_4[[ 'order_id',
                             'product_id',
                             'customer_id',
                             'seller_id',
                             'order_item_id',
                             'price',
                             'freight_value',
                             'product_name_lenght',
                             'product_description_lenght',
                             'product_photos_qty',
                             'product_weight_g',
                             'product_length_cm',
                             'product_height_cm',
                             'product_width_cm',
                             'customer_city',
                             'customer_state',
                             'customer_lat',
                             'customer_lng',
                             'seller_city',
                             'seller_state',
                             'seller_lat',
                             'seller_lng',
                             'order_purchase_timestamp',
                             'order_approved_at',
                             'shipping_limit_date',
                             'order_delivered_carrier_date',
                             'order_delivered_customer_date',
                             'order_estimated_delivery_date']]

In [88]:
data_merge.head(2)

,order_id,product_id,customer_id,seller_id,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,order_purchase_timestamp,order_approved_at,shipping_limit_date,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,1,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0,sao paulo,SP,-23.576983,-46.587161,maua,SP,-23.680729,-46.444238,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-06 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,1,118.70,22.76,29.0,178.0,1.0,400.0,19.0,13.0,19.0,barreiras,BA,-12.177924,-44.660711,belo horizonte,SP,-19.807681,-43.980427,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-30 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00


In [89]:
data_merge['product_volume'] = data_merge['product_length_cm'] * data_merge['product_height_cm'] * data_merge['product_width_cm']

In [90]:
data_merge.drop(['product_width_cm','product_length_cm', 'product_height_cm', 'product_height_cm','product_description_lenght', 'product_name_lenght','product_photos_qty'], axis=1, inplace=True)

In [91]:
data_merge.head(2)

,order_id,product_id,customer_id,seller_id,order_item_id,price,freight_value,product_weight_g,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,order_purchase_timestamp,order_approved_at,shipping_limit_date,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,product_volume
0,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,1,29.99,8.72,500.0,sao paulo,SP,-23.576983,-46.587161,maua,SP,-23.680729,-46.444238,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-06 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1976.0
1,53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,1,118.70,22.76,400.0,barreiras,BA,-12.177924,-44.660711,belo horizonte,SP,-19.807681,-43.980427,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-30 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,4693.0


In [103]:
%%time
data_all_agg = data_merge.groupby(['order_id']).agg({ 'seller_id':'max',
                                                      'order_item_id':'max',
                                                      'price':'sum',
                                                      'freight_value':'max',
                                                      'product_weight_g':'sum',
                                                      'product_volume':'sum',
                                                      'customer_city':'max',
                                                      'customer_state':'max',
                                                      'customer_lat':'max',
                                                      'customer_lng':'max',
                                                      'seller_city':'max',
                                                      'seller_state':'max',
                                                      'seller_lat':'max',
                                                      'seller_lng':'max',
                                                      'order_purchase_timestamp':'max',
                                                      'order_approved_at':'max',
                                                      'shipping_limit_date':'max',
                                                      'order_delivered_carrier_date':'max',
                                                      'order_delivered_customer_date':'max',
                                                      'order_estimated_delivery_date':'max'})

CPU times: user 1min 36s, sys: 2.02 s, total: 1min 38s
Wall time: 1min 35s


In [104]:
data_all_agg.head(2)

,seller_id,order_item_id,price,freight_value,product_weight_g,product_volume,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,order_purchase_timestamp,order_approved_at,shipping_limit_date,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
order_id,,,,,,,,,,,,,,,,,,,,
00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,1,58.9,13.29,650.0,3528.0,campos dos goytacazes,RJ,-21.762775,-41.309633,volta redonda,SP,-22.496953,-44.127492,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,1,239.9,19.93,30000.0,60000.0,santa fe do sul,SP,-20.220527,-50.903424,sao paulo,SP,-23.565096,-46.518565,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-03 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00


In [105]:
# Save our results
data_all_agg.to_csv('data_raw/data_all_agg.csv')

In [113]:
data_all_agg.reset_index(inplace=True)

In [114]:
import numpy as np

In [115]:
data_all_agg['distance'] = np.abs(data_all_agg.customer_lat - data_all_agg.seller_lat) + np.abs(data_all_agg.customer_lng - data_all_agg.seller_lng) 

In [117]:
data_all_agg.head(2)

,order_id,seller_id,order_item_id,price,freight_value,product_weight_g,product_volume,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,order_purchase_timestamp,order_approved_at,shipping_limit_date,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,distance
0,00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,1,58.9,13.29,650.0,3528.0,campos dos goytacazes,RJ,-21.762775,-41.309633,volta redonda,SP,-22.496953,-44.127492,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,3.552037
1,00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,1,239.9,19.93,30000.0,60000.0,santa fe do sul,SP,-20.220527,-50.903424,sao paulo,SP,-23.565096,-46.518565,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-03 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,7.729428


In [111]:
# we are goin to change the time to datetime format
data_all_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94653 entries, 00010242fe8c5a6d1ba2dd792cb16214 to fffe41c64501cc87c801fd61db3f6244
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   seller_id                      94653 non-null  object 
 1   order_item_id                  94653 non-null  int64  
 2   price                          94653 non-null  float64
 3   freight_value                  94653 non-null  float64
 4   product_weight_g               94653 non-null  float64
 5   product_volume                 94653 non-null  float64
 6   customer_city                  94653 non-null  object 
 7   customer_state                 94653 non-null  object 
 8   customer_lat                   94653 non-null  float64
 9   customer_lng                   94653 non-null  float64
 10  seller_city                    94653 non-null  object 
 11  seller_state                   94653 non-null  object

In [126]:
# data_all_agg['delivery_time'] = pd.Timedelta(pd.to_datetime(data_all_agg.order_delivered_customer_date) - pd.to_datetime(data_all_agg.order_purchase_timestamp)).seconds/3600.